# Data Analyis & Processing (CSV to SQLite)

**Source:** Erik Voeten's UN General Assembly Voting Dataset 1946–2023 (Harvard Dataverse)  

## 0. Imports & Paths

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

CSV_PATH = Path('/Users/kerematas/Desktop/database/un-votes/data/UNVotes.csv')
DB_PATH  = Path('/Users/kerematas/Desktop/database/un-votes/un_votes.db')

# Issue flag columns in CSV in (issue_id, display name)
ISSUE_MAP = {
    'me': ('ME', 'Middle East'),
    'nu': ('NU', 'Nuclear Weapons & Materials'),
    'di': ('DI', 'Arms Control & Disarmament'),
    'hr': ('HR', 'Human Rights'),
    'co': ('CO', 'Colonialism'),
    'ec': ('EC', 'Economic Development'),
}

# Numeric vote codes in readable labels
VOTE_MAP = {1: 'YES', 2: 'ABSTAIN', 3: 'NO', 8: 'ABSENT'}

print('Paths OK:', CSV_PATH.exists(), '|', DB_PATH.parent.exists())

Paths OK: True | True


## 1. Load Raw CSV

The Voeten dataset has 26 columns. Key ones we care about:

| Column | Meaning |
|--------|----------|
| `rcid` | Resolution call ID (numeric) |
| `unres` | UN resolution code (e.g. `A/RES/77/272`) |
| `Country` | 3-letter ISO-ish country code |
| `Countryname` | Full country name |
| `vote` | 1=YES, 2=ABSTAIN, 3=NO, 8=ABSENT, 9=NOT A MEMBER |
| `date` | Vote date |
| `session` | UNGA session number |
| `amend` | 1 if this is an amendment vote (procedural) |
| `para` | 1 if this is a paragraph vote (procedural) |
| `descr` | Long description of the resolution |
| `me/nu/di/hr/co/ec` | Binary issue-topic flags |

In [2]:
raw = pd.read_csv(
    CSV_PATH,
    index_col=0,
    parse_dates=['date'],
    low_memory=False,
)

print(f'Shape: {raw.shape[0]:,} rows × {raw.shape[1]} columns')
print(f'Columns: {list(raw.columns)}')

Shape: 1,266,486 rows × 25 columns
Columns: ['rcid', 'ccode', 'member', 'vote', 'Country', 'Countryname', 'session', 'abstain', 'yes', 'no', 'importantvote', 'date', 'unres', 'amend', 'para', 'short', 'descr', 'me', 'nu', 'di', 'hr', 'co', 'ec', 'ident', 'resid']


In [3]:
# Lets peek at the first few rows to understand the raw structure
raw.head(3)

,rcid,ccode,member,vote,Country,Countryname,session,abstain,yes,no,...,short,descr,me,nu,di,hr,co,ec,ident,resid
1,3,2.0,1.0,1,USA,United States of America,1,4.0,29.0,18.0,...,"AMENDMENTS, RULES OF PROCEDURE",TO ADOPT A CUBAN AMENDMENT TO THE UK PROPOSAL ...,0,0,0,0,0,0,0.0,1001
2,3,20.0,1.0,3,CAN,Canada,1,4.0,29.0,18.0,...,"AMENDMENTS, RULES OF PROCEDURE",TO ADOPT A CUBAN AMENDMENT TO THE UK PROPOSAL ...,0,0,0,0,0,0,0.0,1001
3,3,31.0,NaN,9,BHS,Bahamas,1,4.0,29.0,18.0,...,"AMENDMENTS, RULES OF PROCEDURE",TO ADOPT A CUBAN AMENDMENT TO THE UK PROPOSAL ...,0,0,0,0,0,0,0.0,1001


In [4]:
# Vote code distribution in raw data
raw['vote'].value_counts().rename({1:'YES',2:'ABSTAIN',3:'NO',8:'ABSENT',9:'NOT A MEMBER'})

vote
YES             730446
NOT A MEMBER    255406
ABSTAIN         116051
ABSENT           95418
NO               69165
Name: count, dtype: int64

## 2. Filter: Keep Only Substantive Full-Resolution Votes

Three filter steps:

1. **Drop `amend=1`** — amendment votes are procedural, not policy positions
2. **Drop `para=1`** — paragraph votes split a resolution into pieces; we want the whole thing
3. **Drop `vote=9`** — "NOT A MEMBER" rows carry no voting signal

Keeping `vote=8` (ABSENT): diplomatic absence is a deliberate signal, not missing data.

In [5]:
n0 = len(raw)

# full resolutions only
# Use != 1 (not == 0) because in later sessions the dataset leaves
# amend/para as NaN (not 0) for full resolutions — NaN != 1 passes correctly.
df = raw[(raw['amend'] != 1) & (raw['para'] != 1)].copy()
print(f'After dropping amendments/paragraphs: {len(df):>8,} rows  (removed {n0 - len(df):,})')

# valid vote codes only
df = df[df['vote'].isin(VOTE_MAP.keys())].copy()
print(f'After dropping non-member rows:        {len(df):>8,} rows  (removed {n0 - len(df):,} total)')

After dropping amendments/paragraphs: 1,041,859 rows  (removed 224,627)
After dropping non-member rows:         862,619 rows  (removed 403,867 total)


## 3. Clean & Engineer Columns

In [6]:
# Readable vote labels
df['vote_label'] = df['vote'].map(VOTE_MAP)

# Extract year
df['year'] = df['date'].dt.year
df['date'] = df['date'].dt.date.astype(str)

# Normalise country code and resolution ID
df['country_id'] = df['Country'].str.upper().str.strip()
df['res_id']     = df['unres'].str.strip()

# Best available description: prefer long descr, fall back to short
df['description'] = df['descr'].fillna(df['short']).str.strip()

# Drop rows missing country or resolution ID
df = df.dropna(subset=['country_id', 'res_id'])

# Deduplicate: one vote per (country, resolution)
df = df.drop_duplicates(subset=['country_id', 'res_id'])

print(f'Final working dataset: {len(df):,} rows')
print(f'Unique countries:      {df["country_id"].nunique()}')
print(f'Unique resolutions:    {df["res_id"].nunique()}')
print(f'Year range:            {int(df["year"].min())} – {int(df["year"].max())}')

Final working dataset: 843,489 rows
Unique countries:      390
Unique resolutions:    5195
Year range:            1946 – 2023


In [9]:
# Vote distribution after cleaning
dist = df['vote_label'].value_counts().reset_index()
dist.columns = ['Vote', 'Count']
dist['%'] = (dist['Count'] / dist['Count'].sum() * 100).round(1)
dist

,Vote,Count,%
0,YES,629161,74.6
1,ABSTAIN,92195,10.9
2,ABSENT,76224,9.0
3,NO,45909,5.4


**Observation:** ~74% YES rate. The UNGA skews toward consensus. 
The analytically rich signal lives in the NO and ABSTAIN split, which is where country blocs reveal themselves.

## 4. Build Normalised Lookup Tables

We decompose the flat CSV into 5 relational tables.
- countries
- votes
- resolutions
- res_issues
- issues


In [10]:
# ── countries table
countries_df = (
    df[['country_id', 'Countryname']]
    .dropna(subset=['Countryname'])          # a handful of rows have no name
    .drop_duplicates('country_id')
    .rename(columns={'Countryname': 'country_name'})
    .sort_values('country_id')
    .reset_index(drop=True)
)

print(f'countries: {len(countries_df)} rows')
countries_df.head(5)

countries: 198 rows


,country_id,country_name
0,AFG,Afghanistan
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,United Arab Emirates


In [11]:
# ── resolutions table
resolutions_df = (
    df[['res_id', 'session', 'date', 'year', 'description',
        'yes', 'no', 'abstain', 'importantvote']]
    .drop_duplicates('res_id')
    .rename(columns={
        'date': 'res_date',
        'yes': 'total_yes', 'no': 'total_no', 'abstain': 'total_abstain',
        'importantvote': 'important'
    })
    .sort_values('res_date')
    .reset_index(drop=True)
)

print(f'resolutions: {len(resolutions_df)} rows')
resolutions_df.tail(5)

resolutions: 5195 rows


,res_id,session,res_date,year,description,total_yes,total_no,total_abstain,important
5190,A/RES/77/272,77,2023-02-20,2023,Cooperation between the United Nations and the...,112.0,7.0,18.0,NaN
5191,A/RES/77/284,77,2023-04-26,2023,Cooperation between the United Nations and the...,122.0,5.0,18.0,NaN
5192,A/RES/77/293,77,2023-06-07,2023,Status of internally displaced persons and ref...,100.0,9.0,59.0,NaN
5193,A/RES/77/301,77,2023-06-29,2023,Independent Institution on Missing Persons in ...,83.0,11.0,62.0,NaN
5194,A/RES/77/313,77,2023-06-30,2023,Financing of the United Nations Interim Force ...,120.0,3.0,1.0,NaN


In [12]:
# ── issues table
issues_df = pd.DataFrame(
    [{'issue_id': v[0], 'issue_name': v[1]} for v in ISSUE_MAP.values()]
)
issues_df

,issue_id,issue_name
0,ME,Middle East
1,NU,Nuclear Weapons & Materials
2,DI,Arms Control & Disarmament
3,HR,Human Rights
4,CO,Colonialism
5,EC,Economic Development


In [13]:
# ── res_issues table
# Each resolution can belong to multiple issue categories, categories are one-hot encoded.

res_issues_rows = []
for col, (issue_id, _) in ISSUE_MAP.items():
    flagged = df[df[col] == 1][['res_id']].drop_duplicates().copy()
    flagged['issue_id'] = issue_id
    res_issues_rows.append(flagged)

res_issues_df = (
    pd.concat(res_issues_rows, ignore_index=True)
    .drop_duplicates()
    .sort_values(['res_id', 'issue_id'])
    .reset_index(drop=True)
)

print(f'res_issues: {len(res_issues_df)} rows')

# How many resolutions per issue?
res_issues_df.groupby('issue_id').size().rename('# resolutions').reset_index().merge(issues_df)

res_issues: 5325 rows


,issue_id,# resolutions,issue_name
0,CO,859,Colonialism
1,DI,1010,Arms Control & Disarmament
2,EC,708,Economic Development
3,HR,959,Human Rights
4,ME,1024,Middle East
5,NU,765,Nuclear Weapons & Materials


In [14]:
# ── votes table
votes_df = (
    df[['country_id', 'res_id', 'vote_label', 'year']]
    .rename(columns={'vote_label': 'votecast'})
    .drop_duplicates(subset=['country_id', 'res_id'])
    .reset_index(drop=True)
)

print(f'votes: {len(votes_df):,} rows')
votes_df.head(5)

votes: 843,489 rows


,country_id,res_id,votecast,year
0,USA,R/1/79,NO,1946
1,CAN,R/1/79,NO,1946
2,CUB,R/1/79,NO,1946
3,HTI,R/1/79,NO,1946
4,DOM,R/1/79,NO,1946


## 5. Write to SQLite

In [15]:
if DB_PATH.exists():
    DB_PATH.unlink()  # fresh build

conn = sqlite3.connect(DB_PATH)

conn.executescript("""
PRAGMA journal_mode=WAL;
PRAGMA synchronous=NORMAL;

CREATE TABLE countries (
    country_id   TEXT PRIMARY KEY,
    country_name TEXT NOT NULL
);

CREATE TABLE resolutions (
    res_id        TEXT PRIMARY KEY,
    session       INTEGER,
    res_date      TEXT,
    year          INTEGER,
    description   TEXT,
    total_yes     INTEGER,
    total_no      INTEGER,
    total_abstain INTEGER,
    important     INTEGER
);

CREATE TABLE issues (
    issue_id   TEXT PRIMARY KEY,
    issue_name TEXT NOT NULL
);

CREATE TABLE res_issues (
    res_id   TEXT NOT NULL REFERENCES resolutions(res_id),
    issue_id TEXT NOT NULL REFERENCES issues(issue_id),
    PRIMARY KEY (res_id, issue_id)
);

CREATE TABLE votes (
    country_id TEXT NOT NULL REFERENCES countries(country_id),
    res_id     TEXT NOT NULL REFERENCES resolutions(res_id),
    votecast   TEXT NOT NULL CHECK(votecast IN ('YES','NO','ABSTAIN','ABSENT')),
    year       INTEGER,
    PRIMARY KEY (country_id, res_id)
);

CREATE INDEX idx_votes_year     ON votes(year);
CREATE INDEX idx_votes_votecast ON votes(votecast);
CREATE INDEX idx_votes_country  ON votes(country_id);
CREATE INDEX idx_res_issues_res ON res_issues(res_id);
""")

countries_df.to_sql('countries',   conn, if_exists='append', index=False)
issues_df.to_sql('issues',         conn, if_exists='append', index=False)
resolutions_df.to_sql('resolutions', conn, if_exists='append', index=False)
res_issues_df.to_sql('res_issues', conn, if_exists='append', index=False)
votes_df.to_sql('votes',           conn, if_exists='append', index=False)

conn.commit()

db_size_mb = DB_PATH.stat().st_size / 1_048_576
print(f'Database written: {DB_PATH}')
print(f'Size: {db_size_mb:.1f} MB')

Database written: /Users/kerematas/Desktop/database/un-votes/un_votes.db
Size: 76.0 MB


## 6. Sanity-Check Queries

In [16]:
checks = {
    'Total votes':            'SELECT COUNT(*) FROM votes',
    'YES votes':              "SELECT COUNT(*) FROM votes WHERE votecast='YES'",
    'Unique countries':       'SELECT COUNT(*) FROM countries',
    'Unique resolutions':     'SELECT COUNT(*) FROM resolutions',
    'Year range':             'SELECT MIN(year), MAX(year) FROM resolutions',
    'Resolutions w/ issues':  'SELECT COUNT(DISTINCT res_id) FROM res_issues',
    'ME resolutions':         "SELECT COUNT(*) FROM res_issues WHERE issue_id='ME'",
    'HR resolutions':         "SELECT COUNT(*) FROM res_issues WHERE issue_id='HR'",
}

for label, sql in checks.items():
    result = conn.execute(sql).fetchone()
    print(f'{label:30s} → {result}')

Total votes                    → (843489,)
YES votes                      → (629161,)
Unique countries               → (198,)
Unique resolutions             → (5195,)
Year range                     → (1946, 2023)
Resolutions w/ issues          → (3696,)
ME resolutions                 → (1024,)
HR resolutions                 → (959,)


In [18]:
# Resolutions per decade
pd.read_sql("""
    SELECT 
        (year / 10) * 10  AS decade,
        COUNT(*)          AS resolutions
    FROM resolutions
    GROUP BY decade
    ORDER BY decade
""", conn)

,decade,resolutions
0,1940,89
1,1950,230
2,1960,205
3,1970,798
4,1980,1378
5,1990,723
6,2000,734
7,2010,778
8,2020,260


In [19]:
# Top 10 countries by total votes cast
pd.read_sql("""
    SELECT c.country_name, COUNT(*) AS votes_cast
    FROM votes v
    JOIN countries c USING (country_id)
    GROUP BY v.country_id
    ORDER BY votes_cast DESC
    LIMIT 10
""", conn)

,country_name,votes_cast
0,Argentina,5104
1,Australia,5104
2,Belgium,5104
3,Belarus,5104
4,Bolivia (Plurinational State of),5104
5,Brazil,5104
6,Canada,5104
7,Chile,5104
8,Colombia,5104
9,Costa Rica,5104


In [17]:
conn.close()
print('Done. Database is ready.')

Done. Database is ready.
